# Get data into a normalized form (3NF) 

In [8]:
from openpyxl import load_workbook
import csv

xlsx_path = "telco_customers.xlsx"
csv_path = "telco_customers.csv"

wb = load_workbook(xlsx_path, read_only=True)
ws = wb.active

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for i, row in enumerate(ws.iter_rows(values_only=True)):
        if i == 0:
            headers = [str(h).strip() for h in row]
            writer.writerow(headers)
        else:
            writer.writerow(row)

# now we have telco_customers.csv from the XLSX

In [22]:
import sqlite3

# connect to SQLite (in-memory or file-based)
conn = sqlite3.connect('telco.db')
cur = conn.cursor()

In [15]:
# create tables
cur.execute("""
CREATE TABLE Location (
  ZipCode TEXT PRIMARY KEY,
  Country TEXT,
  State TEXT,
  City TEXT,
  Latitude REAL,
  Longitude REAL
);
""")

cur.execute("""
CREATE TABLE Customer (
    CustomerID TEXT PRIMARY KEY,
    Gender TEXT,
    SeniorCitizen INTEGER,
    Partner TEXT,
    Dependents TEXT,
    Churn TEXT,
    ZipCode TEXT,
    ChurnLabel TEXT,
    ChurnValue INTEGER,
    ChurnScore INTEGER,
    CLTV INTEGER,
    ChurnReason TEXT,
    FOREIGN KEY (ZipCode) REFERENCES Location(ZipCode)
);
""")

cur.execute("""
CREATE TABLE Services (
  CustomerID TEXT PRIMARY KEY,
  TenureMonths INTEGER,
  PhoneService TEXT,
  MultipleLines TEXT,
  InternetService TEXT,
  OnlineSecurity TEXT,
  OnlineBackup TEXT,
  DeviceProtection TEXT,
  TechSupport TEXT,
  StreamingTV TEXT,
  StreamingMovies TEXT,
  FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
);
""")

cur.execute("""
CREATE TABLE Billing (
  CustomerID TEXT PRIMARY KEY,
  Contract TEXT,
  PaperlessBilling TEXT,
  PaymentMethod TEXT,
  MonthlyCharges REAL,
  TotalCharges REAL,
  FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
);
""")

In [16]:
# read CSV and insert data
with open('telco_customers.csv', 'r', newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)

    # to avoid inserting duplicates into Location, keep track of seen ZipCodes
    seen_zipcodes = set()

    for row in reader:
        customer_id = row['CustomerID'].strip()

        # prepare location data
        zip_code = str(row['Zip Code']).strip()
        country = row['Country'].strip()
        state = row['State'].strip()
        city = row['City'].strip()
        latitude = float(row['Latitude']) if row['Latitude'] else None
        longitude = float(row['Longitude']) if row['Longitude'] else None

        # insert into Location only if not already inserted
        if zip_code not in seen_zipcodes:
            cur.execute("""
                INSERT OR IGNORE INTO Location (ZipCode, Country, State, City, Latitude, Longitude)
                VALUES (?, ?, ?, ?, ?, ?);
            """, (zip_code, country, state, city, latitude, longitude))
            seen_zipcodes.add(zip_code)

        # prepare customer data
        gender = row['Gender'].strip()
        senior_map = {'yes': 1, 'no': 0, '': 0, None: 0}
        senior_citizen = senior_map.get(row['Senior Citizen'].strip().lower(), 0)
        partner = row['Partner'].strip()
        dependents = row['Dependents'].strip()
        churn_label = row['Churn Label'].strip()
        churn_value = int(row['Churn Value']) if row['Churn Value'] else 0
        churn_score = int(row['Churn Score']) if row['Churn Score'] else 0
        cltv = int(row['CLTV']) if row['CLTV'] else None
        churn_reason = row['Churn Reason'].strip() if row['Churn Reason'] else ''

        cur.execute("""
            INSERT OR REPLACE INTO Customer (
                CustomerID, Gender, SeniorCitizen, Partner, Dependents,
                ChurnLabel, ChurnValue, ChurnScore, CLTV, ChurnReason, ZipCode
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """, (customer_id, gender, senior_citizen, partner, dependents, churn_label, churn_value, churn_score, cltv, churn_reason, zip_code))

        # prepare services data 
        tenure_months = int(row['Tenure Months']) if row['Tenure Months'] else None
        phone_service = row['Phone Service'].strip()
        multiple_lines = row['Multiple Lines'].strip()
        internet_service = row['Internet Service'].strip()
        online_security = row['Online Security'].strip()
        online_backup = row['Online Backup'].strip()
        device_protection = row['Device Protection'].strip()
        tech_support = row['Tech Support'].strip()
        streaming_tv = row['Streaming TV'].strip()
        streaming_movies = row['Streaming Movies'].strip()

        cur.execute("""
            INSERT OR REPLACE INTO Services (
                CustomerID, TenureMonths, PhoneService, MultipleLines, InternetService,
                OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """, (customer_id, tenure_months, phone_service, multiple_lines, internet_service,
              online_security, online_backup, device_protection, tech_support, streaming_tv, streaming_movies))

        # prepare billing data
        contract = row['Contract'].strip()
        paperless_billing = row['Paperless Billing'].strip()
        payment_method = row['Payment Method'].strip()
        monthly_charges = float(row['Monthly Charges']) if row['Monthly Charges'] else None
        total_charges_raw = row['Total Charges'].strip()
        total_charges = float(total_charges_raw) if total_charges_raw.replace('.', '', 1).isdigit() else None

        cur.execute("""
            INSERT OR REPLACE INTO Billing (
                CustomerID, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges
            ) VALUES (?, ?, ?, ?, ?, ?);
        """, (customer_id, contract, paperless_billing, payment_method, monthly_charges, total_charges))

# commit changes
conn.commit()

In [23]:
import pandas as pd

# JOIN tables and load into Pandas DataFrame for analysis
query = """
SELECT
    c.CustomerID,
    c.Gender,
    c.SeniorCitizen,
    c.Partner,
    c.Dependents,
    c.ChurnLabel,
    c.ChurnValue,
    c.ChurnScore,
    c.CLTV,
    c.ChurnReason,
    l.Country,
    l.State,
    l.City,
    l.ZipCode,
    l.Latitude,
    l.Longitude,
    s.TenureMonths,
    s.PhoneService,
    s.MultipleLines,
    s.InternetService,
    s.OnlineSecurity,
    s.OnlineBackup,
    s.DeviceProtection,
    s.TechSupport,
    s.StreamingTV,
    s.StreamingMovies,
    b.Contract,
    b.PaperlessBilling,
    b.PaymentMethod,
    b.MonthlyCharges,
    b.TotalCharges
FROM Customer c
JOIN Location l ON c.ZipCode = l.ZipCode
JOIN Services s ON c.CustomerID = s.CustomerID
JOIN Billing b ON c.CustomerID = b.CustomerID;
"""

normalized_df = pd.read_sql_query(query, conn)
conn.close()

In [24]:
normalized_df

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,ChurnLabel,ChurnValue,ChurnScore,CLTV,ChurnReason,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,3668-QPYBK,Male,0,No,No,Yes,1,86,3239,Competitor made better offer,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
1,9237-HQITU,Female,0,No,Yes,Yes,1,67,2701,Moved,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65
2,9305-CDSKC,Female,0,No,Yes,Yes,1,86,5372,Moved,...,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50
3,7892-POOKP,Female,0,Yes,Yes,Yes,1,84,5003,Moved,...,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05
4,0280-XJGEX,Male,0,No,Yes,Yes,1,89,5340,Competitor had better devices,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Female,0,No,No,No,0,45,5306,,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.40
7039,6840-RESVB,Male,0,Yes,Yes,No,0,59,2140,,...,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50
7040,2234-XADUH,Female,0,Yes,Yes,No,0,71,5560,,...,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90
7041,4801-JZAZL,Female,0,Yes,Yes,No,0,59,2793,,...,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45


In [25]:
def report(df):
    cols = []
    data_types = []
    unique_values = []
    n_unique_values = []

    for col in df.columns:
        cols.append(col)
        data_types.append(df[col].dtypes)
        unique_values.append(df[col].unique()[:5])
        n_unique_values.append(df[col].nunique())

    return pd.DataFrame({'Column': cols, 'data_type': data_types, 'unique_values': unique_values, 'n_unique_values': n_unique_values})

In [26]:
report(normalized_df)

,Column,data_type,unique_values,n_unique_values
0,CustomerID,object,"[3668-QPYBK, 9237-HQITU, 9305-CDSKC, 7892-POOK...",7043
1,Gender,object,"[Male, Female]",2
2,SeniorCitizen,int64,"[0, 1]",2
3,Partner,object,"[No, Yes]",2
4,Dependents,object,"[No, Yes]",2
5,ChurnLabel,object,"[Yes, No]",2
6,ChurnValue,int64,"[1, 0]",2
7,ChurnScore,int64,"[86, 67, 84, 89, 78]",85
8,CLTV,int64,"[3239, 2701, 5372, 5003, 5340]",3438
9,ChurnReason,object,"[Competitor made better offer, Moved, Competit...",21


The *Lat Lon* and *Count* column are dropped because they do not add any value to our experiment.

In [27]:
normalized_df.to_csv("normalized_data.csv", index=False)